In [40]:
import requests
import pandas as pd
from datetime import datetime

In [42]:
available_tickers_df = pd.read_csv('available_tickers.csv')
available_tickers_df.head(5)

,Ticker
0,INDF
1,FLIC
2,NBHC
3,GRVY
4,GPK


In [43]:
available_tickers = available_tickers_df['Ticker'].tolist()

In [50]:
import requests
import pandas as pd
from datetime import datetime

# Function to make API request and get data for a specific date range
def get_market_cap_data(ticker, start_date, end_date, api_key):
    try:
        url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}"
        params = {
            "limit": 10000,
            "from": start_date,
            "to": end_date,
            "apikey": api_key
        }
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an error for bad status codes
        data = response.json()
        
        # If the data is not in the expected format, handle it
        if isinstance(data, dict) and 'historical' in data:
            data = data['historical']
        
        if not data or not isinstance(data, list):
            print(f"No data returned for {ticker} between {start_date} and {end_date}")
            return pd.DataFrame()
        
        df = pd.DataFrame(data)
        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {ticker}: {e}")
        return pd.DataFrame()

# Function to split date range into sub-ranges
def split_date_range(start_date, end_date, years_per_split=5):
    date_ranges = []
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    while start < end:
        split_end = min(start.replace(year=start.year + years_per_split), end)
        date_ranges.append((start.strftime("%Y-%m-%d"), split_end.strftime("%Y-%m-%d")))
        start = split_end
        
    return date_ranges

# Function to process a list of tickers
def process_tickers(tickers, start_date, end_date, api_key):
    all_data = []
    
    for ticker in tickers:
        print(f"Processing {ticker}...")
        date_ranges = split_date_range(start_date, end_date)
        df_list = []
        
        for date_range in date_ranges:
            df = get_market_cap_data(ticker, date_range[0], date_range[1], api_key)
            if not df.empty:
                df_list.append(df)
        
        if df_list:
            # Combine all dataframes for the ticker
            combined_df = pd.concat(df_list)
            combined_df['date'] = pd.to_datetime(combined_df['date'])
            combined_df.set_index('date', inplace=True)
            
            # Ensure all days are covered
            full_index = pd.date_range(start=start_date, end=end_date, freq='D')
            combined_df = combined_df.reindex(full_index)
            
            # Ensure only numeric data is used for calculations
            numeric_df = combined_df.select_dtypes(include=['number'])
            
            # Calculate monthly average market cap
            monthly_avg = numeric_df.resample('M').mean().round(2)
            
            # Reset index and rename columns for final output
            monthly_avg.reset_index(inplace=True)
            monthly_avg.rename(columns={'marketCap': 'Monthly Avg Market Cap', 'index': 'Date'}, inplace=True)
            
            # Add the ticker column after calculating averages
            monthly_avg['Ticker'] = ticker
            
            all_data.append(monthly_avg[['Ticker', 'Date', 'Monthly Avg Market Cap']])
    
    # Combine all tickers' data
    if all_data:
        final_df = pd.concat(all_data)
    else:
        final_df = pd.DataFrame(columns=['Ticker', 'Date', 'Monthly Avg Market Cap'])
    
    return final_df

# Main function
def main():
    api_key = "bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL"
    tickers = available_tickers  # list of tickers
    start_date = "2010-01-01"
    end_date = "2024-06-30"
    
    final_df = process_tickers(tickers, start_date, end_date, api_key)
    
    # Save to CSV
    final_df.to_csv("historical_market_cap.csv", index=False)
    return final_df

# Call the main function and capture final_df
final_df = main()

# Now you can display final_df
final_df


Processing INDF...
Request failed for INDF: 429 Client Error: Too Many Requests for url: https://financialmodelingprep.com/api/v3/historical-market-capitalization/INDF?limit=10000&from=2010-01-01&to=2015-01-01&apikey=bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL
Request failed for INDF: 429 Client Error: Too Many Requests for url: https://financialmodelingprep.com/api/v3/historical-market-capitalization/INDF?limit=10000&from=2015-01-01&to=2020-01-01&apikey=bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL
Request failed for INDF: 429 Client Error: Too Many Requests for url: https://financialmodelingprep.com/api/v3/historical-market-capitalization/INDF?limit=10000&from=2020-01-01&to=2024-06-30&apikey=bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL
Processing FLIC...
Request failed for FLIC: 429 Client Error: Too Many Requests for url: https://financialmodelingprep.com/api/v3/historical-market-capitalization/FLIC?limit=10000&from=2010-01-01&to=2015-01-01&apikey=bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL
Request failed for FLIC: 429 Client Er

KeyboardInterrupt: 

In [34]:


# Function to make API request and get data for a specific date range
def get_market_cap_data(ticker, start_date, end_date, api_key):
    url = f"https://financialmodelingprep.com/api/v3/historical-market-capitalization/{ticker}"
    params = {
        "limit": 10000,
        "from": start_date,
        "to": end_date,
        "apikey": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data)
    return df

# Function to split date range into sub-ranges
def split_date_range(start_date, end_date, years_per_split=5):
    date_ranges = []
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    while start < end:
        split_end = min(start.replace(year=start.year + years_per_split), end)
        date_ranges.append((start.strftime("%Y-%m-%d"), split_end.strftime("%Y-%m-%d")))
        start = split_end
        
    return date_ranges

# Function to process a list of tickers
def process_tickers(tickers, start_date, end_date, api_key):
    all_data = []
    
    for ticker in tickers:
        print(f"Processing {ticker}...")
        date_ranges = split_date_range(start_date, end_date)
        df_list = []
        
        for date_range in date_ranges:
            df = get_market_cap_data(ticker, date_range[0], date_range[1], api_key)
            df_list.append(df)
        
        # Combine all dataframes for the ticker
        combined_df = pd.concat(df_list)
        combined_df['date'] = pd.to_datetime(combined_df['date'])
        combined_df.set_index('date', inplace=True)
        
        full_index = pd.date_range(start=start_date, end=end_date, freq='D')
        combined_df = combined_df.reindex(full_index)

        
        # Ensure only numeric data is used for calculations
        numeric_df = combined_df.select_dtypes(include=['number'])
        
        # Calculate monthly average market cap
        monthly_avg = numeric_df.resample('M').mean().round(2)
        
        # Add the ticker column after calculating averages
        monthly_avg['ticker'] = ticker
        
        all_data.append(monthly_avg)
    
    # Combine all tickers data
    final_df = pd.concat(all_data)
    return final_df

# Main function
def main():
    api_key = "bPdPob02ZdQAvgbuhxpPqHwX6UQjM8iL"
    tickers = ["INDF"]  # Replace with your list of tickers
    start_date = "2010-01-01"
    end_date = "2024-06-30"
    
    final_df = process_tickers(tickers, start_date, end_date, api_key)
    
    # Save to CSV
    final_df.to_csv("historical_market_cap.csv")
    return final_df

# Call the main function and capture final_df
final_df = main()

# Now you can display final_df
final_df

Processing INDF...


,marketCap,ticker
2010-01-31,NaN,INDF
2010-02-28,NaN,INDF
2010-03-31,NaN,INDF
2010-04-30,NaN,INDF
2010-05-31,NaN,INDF
...,...,...
2024-02-29,7910489.16,INDF
2024-03-31,7987231.35,INDF
2024-04-30,8128784.05,INDF
2024-05-31,8202635.49,INDF


In [18]:
final_df.head(100)

,marketCap,ticker
2010-01-31,1.793956e+08,FLIC
2010-02-28,1.723207e+08,FLIC
2010-03-31,1.815692e+08,FLIC
2010-04-30,1.789889e+08,FLIC
2010-05-31,1.874079e+08,FLIC
2010-06-30,1.819694e+08,FLIC
2010-07-31,2.137590e+08,FLIC
2010-08-31,2.128129e+08,FLIC
2010-09-30,2.083490e+08,FLIC
2010-10-31,2.175888e+08,FLIC
